In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras.layers import GRU
from tensorflow.keras.regularizers import l2

from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dense, Lambda, Layer
from tensorflow.keras.models import Model

In [ ]:
acp = pd.read_excel("acp.fasta.xlsx")
acp['seq_Length'] = acp['sequence'].apply(lambda x: len(x))
acp

In [ ]:
non_acp= pd.read_excel("non-acp.fasta.xlsx")
non_acp['seq_Length'] = non_acp['sequence'].apply(lambda x: len(x))
non_acp

In [ ]:
whole_seq= pd.concat([acp,non_acp])
whole_seq

In [ ]:
whole_seq['sequence'].duplicated().any()

In [ ]:
whole_seq = whole_seq.drop_duplicates(subset='sequence', keep='first')
whole_seq

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Amino acid encoding
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_idx = {aa: i for i, aa in enumerate(amino_acids)}

# Encode sequences
all_sequences = whole_seq['sequence'].tolist()

# Filter sequences shorter than 85 amino acids
sequences = [seq for seq in all_sequences if len(seq) <= 85]
# Create indices for splitting
indices = np.arange(len(sequences))
train_indices, test_indices = train_test_split(
    indices,
    test_size=0.03,
    random_state=42
)

# Further split train_indices into train_indices and validation_indices
train_indices, validation_indices = train_test_split(
    train_indices,
    test_size=0.1,  # Adjust the validation set size as needed
    random_state=42
)

# Split sequences based on indices
training_sequences = [sequences[i] for i in train_indices]
validation_sequences = [sequences[i] for i in validation_indices]
testing_sequences = [sequences[i] for i in test_indices]

# Find the maximum sequence length across all sets
max_len = max(max(len(seq) for seq in training_sequences),
              max(len(seq) for seq in validation_sequences),
              max(len(seq) for seq in testing_sequences))

# Function to pad sequences and create masks
def process_sequences(sequences):
    encoded_seqs = [[aa_to_idx[aa] for aa in seq] for seq in sequences]
    padded_seqs = tf.keras.preprocessing.sequence.pad_sequences(encoded_seqs, padding='post', maxlen=max_len)
    mask = tf.cast(padded_seqs != 0, dtype=tf.float32)  # 0 represents padding, 1 represents actual data
    one_hot_seqs = tf.one_hot(padded_seqs, len(amino_acids))
    return one_hot_seqs, mask

# Process sequences for each set
train_sequences, train_mask = process_sequences(training_sequences)
valid_sequences, valid_mask = process_sequences(validation_sequences)
test_sequences, test_mask = process_sequences(testing_sequences)

# Print shapes of the resulting tensors
print("Training sequences shape:", train_sequences.shape)
print("Validation sequences shape:", valid_sequences.shape)
print("Testing sequences shape:", test_sequences.shape)
print("Training mask shape:", train_mask.shape)
print("Validation mask shape:", valid_mask.shape)
print("Testing mask shape:", test_mask.shape)

In [ ]:
import subprocess

def get_gpu_info():
    cmd = 'nvidia-smi --query-gpu=utilization.gpu,memory.used --format=csv,noheader,nounits'
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, text=True)
    gpu_info = result.stdout.strip().split(', ')
    return gpu_info

gpu_utilization, gpu_memory_used = get_gpu_info()
print(f"GPU Utilization: {gpu_utilization}%")
print(f"GPU Memory Used: {gpu_memory_used} MB")


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*Gradients do not exist for variables.*")


In [ ]:
# this doesn't show warning every time so it might help prevent the jupyter from crashing.
import tensorflow as tf
tf.get_logger().setLevel('ERROR')


# VAE model with deeper GRU layers

# Loss instaiblity

Here are a few suggestions to improve your VAE model for generating peptide sequences:

- Since most of your sequences are shorter than 85 amino acids, you could reduce the max sequence length (max_len) to something like 100 instead of 207. This will allow the model to better focus on the length distribution seen in the training data. (done)

- Try using 1D convolutional layers before or after the GRU layers. Conv1D layers can help learn local patterns and motifs in the sequences. (done)

- Add dropout layers after the GRU and dense layers to prevent overfitting. A dropout rate of 0.2-0.5 is commonly used.

- Reduce the latent space dimension (latent_dim) to 32 or 48. This will regularize the model. You can then gradually increase it if needed. (done)

- Use a recurrent layer like LSTM instead of GRU if you want to better capture long-range dependencies. 

- Try a shallower and wider architecture rather than deep and narrow. For example, use only 1-2 GRU layers with 256-512 units rather than 4 layers of 64 units.

- Use batch normalization and layer normalization to stabilize training.

- Use a learning rate scheduler like cosine decay to slowly reduce the LR over epochs.(done)

- Use a larger batch size if possible to smooth out noise during training.

The key is to experiment with different architectures, optimizers, regularizers and hyperparameters to find the right balance for your specific sequence generation task. Start simple and modify incrementally.

The loss value you're observing in the training process (1.3822e-05) seems to be extremely low and doesn't make sense, especially if your validation loss (val_loss) is negative. This could be indicative of a training issue. A few things to check:

- Scaling Issues: Make sure your input data is properly scaled. Extremely small values can lead to numerical instability and unexpected results in training.

- Loss Function: Ensure that your loss function is properly defined and calculated. Check if the Wasserstein loss and any other loss components are being computed correctly.

- Learning Rate: The learning rate and decay rate in your optimizer might need adjustments. A learning rate that's too high or too low can impact convergence.

- Gradient Clipping: If your gradients are exploding, it can lead to unusual loss values. Consider adding gradient clipping to prevent this.

- Batch Size: Smaller batch sizes can lead to faster convergence in some cases.

- Regularization: Ensure you're not using excessive regularization, which could lead to overly small weight updates.

- Data Preprocessing: Verify that your data preprocessing is consistent, especially with regard to padding and sequence lengths.

- Debugging Outputs: Add additional print or logging statements to monitor the values of the loss components and other key variables during training.

- Model Architecture: Sometimes, extreme reductions in loss can be indicative of a model that's too large or too powerful for the task. Consider simplifying your model architecture further.

- Evaluation Metric: Ensure you're using appropriate evaluation metrics for your task. Negative validation loss might indicate a problem with the evaluation metric.

- Given the complexity of the issue, it might be helpful to try several debugging steps and experiment with different approaches to narrow down the cause of the unusual loss behavior.







In [ ]:
device = "gpu"
# Enable eager execution
tf.config.run_functions_eagerly(True)

# VAE Model
latent_dim = 32

class Sampling(layers.Layer):
  # Reparameterization trick
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon



encoder_inputs = tf.keras.Input(shape=(max_len,))
x = layers.Embedding(len(amino_acids), latent_dim)(encoder_inputs)
x = layers.Dense(latent_dim)(x)

# Add bidirectional GRU with simpler units
x = layers.Bidirectional(layers.GRU(128, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(128))(x)
# Reshape the output for the next GRU layer
x = layers.Reshape((-1, 256))(x)
x = layers.GRU(256, kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)

x = layers.BatchNormalization()(x)
z_mean = layers.Dense(latent_dim, name='z_mean', kernel_regularizer=l2(0.01))(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var', kernel_regularizer=l2(0.01))(x)
z = Sampling()([z_mean, z_log_var])
encoder = tf.keras.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')



class Decoder(tf.keras.Model):
    def __init__(self, latent_dim, max_len):
        super(Decoder, self).__init__()
        self.latent_dim = latent_dim
        self.max_len = max_len

        self.gru1 = layers.GRU(128, return_sequences=True)
        self.gru2 = layers.GRU(128, return_sequences=True)
        self.dense = layers.Dense(256, kernel_regularizer=l2(0.01))
        self.output_layer = layers.Dense(len(amino_acids), activation='softmax', kernel_regularizer=l2(0.01))

    def call(self, inputs, sequence_length):
        reshaped_inputs = tf.expand_dims(inputs, axis=1)
        repeated_inputs = tf.repeat(reshaped_inputs, sequence_length, axis=1)

        x = self.gru1(repeated_inputs)
        x = self.dense(x)
        decoded_output = self.output_layer(x)

        return decoded_output

decoder = Decoder(latent_dim, max_len)





# Update the VAE model to use the custom WassersteinLossLayer
class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder


    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructions = self.decoder(z, sequence_length=max_len)
        kl_loss = - 0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return reconstructions


vae = VAE(encoder, decoder)


# Reshape train and test sequences
train_sequences = tf.reshape(train_sequences, (-1, max_len))
test_sequences = tf.reshape(test_sequences, (-1, max_len))
valid_sequences = tf.reshape(valid_sequences, (-1, max_len))



# Define the learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9
)

# Use Adagrad optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=1.0)
vae.compile(optimizer=optimizer)



# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=10, restore_best_weights=True
# )


history_vae = vae.fit(
    train_sequences, train_sequences,
    epochs=80,
    batch_size = 32,
    validation_data=(valid_sequences, valid_sequences),
    verbose=1

)

In [ ]:
import matplotlib.pyplot as plt

# Access training and validation losses
train_loss = history_vae.history['loss']
# val_loss = history_vae.history['val_loss']

# Plot training and validation losses
plt.plot(train_loss, label='Train Loss')
# plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
def generate_sequences_with_varying_lengths(vae_model, sequences, masks):
    generated_sequences = []

    for i, (seq, mask) in enumerate(zip(sequences, masks)):
        seq_length = int(tf.reduce_sum(mask))  # Get the actual length of the sequence based on the mask
        # Sample a latent vector from a normal distribution
        sampled_z = np.random.normal(size=(1, latent_dim))

        # Decode the sampled latent vector to generate a sequence with the actual length
        generated_sequence_indices = vae_model.decoder(sampled_z, sequence_length=seq_length)
        
        # Convert generated sequence indices to amino acid letters
        generated_sequence = ''
        for idx_seq in generated_sequence_indices:
            subsequence = [amino_acids[int(np.argmax(probs))] for probs in idx_seq]
            generated_sequence += ''.join(subsequence)
        
        # Append the generated sequence to the list
        generated_sequences.append(generated_sequence)

    return generated_sequences

# Generate sequences with varying lengths from the test set
generated_sequences = generate_sequences_with_varying_lengths(vae, test_sequences, test_mask)


In [ ]:
import Levenshtein

for s1, s2 in zip(generated_sequences, testing_sequences):
    distance = Levenshtein.distance(s1, s2)
    print(f"Levenshtein distance between '{s1}' and '{s2}': {distance}")
